In [2]:
from brainbox.task import trials
from one.api import ONE
import pandas as pd
import numpy as np

import time

start=time.time()
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
eid = '58b1e920-cfc8-467e-b28b-7654a55d0977'


def get_raster_and_psth(eid):
    #Load trials into df
    dat = one.load_dataset(eid, 'alf/_ibl_trials.table.pqt')
    df=pd.DataFrame(dat)
    #Choose one type of event
    trials_=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)].index
    #df=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)]
    events=df["stimOn_times"]
    #Get spike_times
    spike_times=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.times.npy')
    #Get clusters
    spike_clusters=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.clusters.npy')
    print(len(spike_clusters))
    print(spike_times.shape)
    #Get binned rasters
    raster_dict={}
    time_vec={}
    for n in list(np.unique(spike_clusters)):
        neuron=np.where(spike_clusters==n)
        if len(list(neuron[0]))!=0:
            #print(len(list(neuron[0])))
            raster=trials.get_event_aligned_raster(spike_times[neuron], events, tbin=0.1, values=None, epoch=[-0.5, 2], bin=True)
            #print(raster)
            if np.isnan(raster[0]).any()==False:
                raster_dict[n]=raster[0]
        time_vec=raster[1]
    psth_dict={}
    for k in raster_dict.keys():
        psth=trials.get_psth(raster_dict[k],trial_ids=trials_)
        psth_dict[k]=psth
    return raster_dict,psth_dict,time_vec

raster_dict, psth_dict,time_vec=get_raster_and_psth(eid)
end=time.time()
print(end-start)

5469038
(5469038,)
4.898306846618652
